# Test of bound and algorithm

In [1]:
import numpy as np
import gym

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.algorithm.backward_feature_selection import BackwardFeatureSelector
from src.algorithm.info_theory.entropy import LeveOneOutEntropyEstimator, NNEntropyEstimator
from src import wenvs

/home/daibak/.virtualenvs/feature_sel/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
est = NNEntropyEstimator()

In [5]:
env = gym.make('CartPole-v1')
env = wenvs.WrapperEnv(env, continuous_state=True)
from src.envs import lqgNdim, cartpole
wenv = wenvs.WrapperEnv(env, n_fake_features=1, n_fake_actions=1, continuous_state=True)
np.random.seed(0)
wenv.seed(0)

num_ep = 1000
trajectories = [np.hstack(wenv.run_episode()) for _ in range (num_ep)]

In [6]:
fs = BackwardFeatureSelector(est, [t for t in trajectories if len(t)>=10])
fs.selectOnError(10, 0.9, 1)

{0, 1, 2, 4, 6}

In [7]:
fs.computeError()

1.0113339562391774

## LQG n-dim

In [8]:
from src.envs import lqgNdim

In [9]:
env = lqgNdim.LQG_nD(0.99, n_dim=2)
wenv = wenvs.WrapperEnv(env, continuous_state=True, continuous_actions=True)

In [10]:
hist = wenv.run_episode(render=True)
wenv.close()

In [11]:
K = env.computeOptimalK()
pi_opt = lambda x: np.clip(K@x, -env.max_action, env.max_action)

In [12]:
hist = wenv.run_episode(policy=pi_opt)
hist[2].sum()

-34.846069838517174

## Infinite CartPole

In [13]:
from src.envs import cartpole

In [14]:
env = cartpole.CartPoleInfinite()
wenv = wenvs.WrapperEnv(env, continuous_state=True)

In [15]:
wenv.run_episode(render=True)
wenv.close()